In [5]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks


ans = pipeline(
    Tasks.acoustic_noise_suppression,
    model='damo/speech_frcrn_ans_cirm_16k')
result = ans(
    'https://modelscope.oss-cn-beijing.aliyuncs.com/test/audios/speech_with_noise1.wav',
    output_path='output.wav')

2023-09-02 15:48:43,485 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.1



Downloading: 100%|██████████| 1.45k/1.45k [00:00<00:00, 495kB/s]






Downloading: 100%|██████████| 177k/177k [00:00<00:00, 991kB/s] 






Downloading: 100%|██████████| 88.2k/88.2k [00:00<00:00, 818kB/s]
























Downloading: 100%|██████████| 55.3M/55.3M [02:54<00:00, 333kB/s]



Downloading: 100%|██████████| 12.8k/12.8k [00:00<00:00, 310kB/s]






Downloading: 100%|██████████| 75.0k/75.0k [00:00<00:00, 332kB/s]






Downloading: 100%|██████████| 152k/152k [00:00<00:00, 1.07MB/s]
2023-09-02 15:51:41,945 - modelscope - INFO - initiate model from /home/lihongji/.cache/modelscope/hub/damo/speech_frcrn_ans_cirm_16k
2023-09-02 15:51:41,945 - modelscope - INFO - initiate model from location /home/lihongji/.cache/modelscope/hub/damo/speech_frcrn_ans_cirm_16k.
2023-09-02 15:51:41,947 - modelscope - INFO - initialize model from /home/lihongji/.cache/modelscope/hub/da

inputs:(1, 75165)
padding: 27165
inputs after padding:(1, 102330)


In [ ]:
# training process
# should modify the local_path
# the dataset of ICASSP_2021_DNS_Challenge has a sized of 230G
import os

from datasets import load_dataset

from modelscope.metainfo import Trainers
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import to_segment

tmp_dir = './checkpoint'
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

hf_ds = load_dataset(
    '/your_local_path/ICASSP_2021_DNS_Challenge',
    'train',
    split='train')
mapped_ds = hf_ds.map(
    to_segment,
    remove_columns=['duration'],
    num_proc=8,
    batched=True,
    batch_size=36)
mapped_ds = mapped_ds.train_test_split(test_size=3000)
mapped_ds = mapped_ds.shuffle()
dataset = MsDataset.from_hf_dataset(mapped_ds)

kwargs = dict(
    model='your_local_path/speech_frcrn_ans_cirm_16k',
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    work_dir=tmp_dir)
trainer = build_trainer(
    Trainers.speech_frcrn_ans_cirm_16k, default_args=kwargs)
trainer.train()

In [ ]:
# evaluation process
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks


ans = pipeline(
    Tasks.acoustic_noise_suppression,
    model='/your_local_path/speech_frcrn_ans_cirm_16k')
result = ans(
    'https://modelscope.oss-cn-beijing.aliyuncs.com/test/audios/speech_with_noise.wav',
    output_path='output.wav')